In [1]:
from wabi import sql_wabilytics, gSheets, maps_2b
import pandas as pd
import wabi
from gspread_dataframe import set_with_dataframe
import boto3
import io
from datetime import date, timedelta, datetime
import geopandas as gpd
import fiona
import time
import json

C:\Users\camilafernandez_wabi\Documents\Notebooks


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wab

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)


In [2]:
#sería el día cero
start = date.today() - timedelta(days =60)

In [3]:
print(start)

2022-09-18


In [4]:
#acá formateamos la fecha para filtrarlo bien en la query
date_time_str = start.strftime("%Y-%m-%d") 
date_time_str = date_time_str +" "+ "00:00:00" 

In [5]:
supplier_orders_query = '''
SELECT
so.supplier_order_id,
so.created_at,
so.delivery_lon,
so.customer_id,
so.delivery_lat,
soi.product_id,
soi.ean,
soi.name,
soi.brand_id,
soi.category_id,
soi.manufacturer,
soi.quantity,
soi.partial_quantity,
soi.units,
s.country_id,
soi.price,
soi.amount,
pd.category, 
b.brand_name,
c.currency
FROM wabi2bOrders.supplier_order so
LEFT JOIN wabi2b.suppliers s 
ON so.supplier_id = s.supplierId 
LEFT JOIN wabi2b.supplier_order_items soi
ON soi.supplier_order_id = so.supplier_order_id
LEFT JOIN wabi2b.country c
ON c.country_id = s.country_id 
LEFT JOIN wabi2b.product_details pd
ON soi.product_id = pd.product_id 
LEFT JOIN wabi2b.brand b 
ON pd.brand_id = b.brand_id
WHERE so.canceled_at IS NULL and so.created_at >= "'''+date_time_str+'''"

'''

In [6]:
supplier_order_df_query = sql_wabilytics.querySQL(supplier_orders_query,'wabi2b')

In [7]:
supplier_order_df = supplier_order_df_query.copy(deep = True)

In [8]:
supplier_order_df

,supplier_order_id,created_at,delivery_lon,customer_id,delivery_lat,product_id,ean,name,brand_id,category_id,manufacturer,quantity,partial_quantity,units,country_id,price,amount,category,brand_name,currency
0,2637281,2022-11-15 01:18:50,106.700533,408215,10.794840,106912,58934588193052,Thùng Nước Cam Ép Twister 24 Chai x 455ML,5324,4752.0,PepsiCo,1.0,1.0,1.0,vn,188000.0,188000.0,"[""Nước trái cây"",""Đồ uống không cồn"",""Đồ uống""]",Twister,VND
1,2637281,2022-11-15 01:18:50,106.700533,408215,10.794840,109169,18935001214233,Thùng Trà bí đao WONDERFARM 310ML X 24 LON,5327,4754.0,Interfood,2.0,2.0,1.0,vn,146000.0,292000.0,"[""Trà"",""Đồ uống không cồn"",""Đồ uống""]",Wonderfarm,VND
2,2637281,2022-11-15 01:18:50,106.700533,408215,10.794840,109202,18935049501623,Thùng Sữa Trái Cây Nutriboost Hương Dâu 24 Cha...,5382,4767.0,Coke Marketplace,1.0,1.0,1.0,vn,171000.0,171000.0,"[""Sữa nước"",""Sữa và sản phẩm làm từ sữa""]",Nutriboost,VND
3,2637281,2022-11-15 01:18:50,106.700533,408215,10.794840,109207,18935049501509,Thùng Sữa Trái Cây Nutriboost Hương Cam 24 Cha...,5382,4767.0,Coke Marketplace,1.0,1.0,1.0,vn,171000.0,171000.0,"[""Sữa nước"",""Sữa và sản phẩm làm từ sữa""]",Nutriboost,VND
4,2637281,2022-11-15 01:18:50,106.700533,408215,10.794840,110466,HRC-8934588873688,HRC-Thùng Trà Ô long Tea Plus 24 Chai x 455ml,5317,4754.0,PepsiCo,2.0,2.0,1.0,vn,168000.0,336000.0,"[""Trà"",""Đồ uống không cồn"",""Đồ uống""]",Olong Tea Plus,VND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606435,2572282,2022-11-04 20:11:22,-103.753485,7881,19.248535,20707,7501005102728,Leche de Soya Ades Natural 946 Ml (IVA incluido),2693,572.0,COCA-COLA,6.0,6.0,1.0,mx,21.5,129.0,"[""Jugos & Néctares"",""Bebidas sin Alcohol "",""Be...",ADES,MXN
2606436,2572282,2022-11-04 20:11:22,-103.753485,7881,19.248535,20707,7501005102728,Leche de Soya Ades Natural 946 Ml (IVA incluido),2693,572.0,COCA-COLA,6.0,6.0,1.0,mx,21.5,129.0,"[""Jugos & Néctares"",""Bebidas sin Alcohol "",""Be...",ADES,MXN
2606437,2572282,2022-11-04 20:11:22,-103.753485,7881,19.248535,20707,7501005102728,Leche de Soya Ades Natural 946 Ml (IVA incluido),2693,572.0,COCA-COLA,6.0,6.0,1.0,mx,21.5,129.0,"[""Jugos & Néctares"",""Bebidas sin Alcohol "",""Be...",ADES,MXN
2606438,2364852,2022-09-27 07:38:17,105.761878,407107,10.040417,170387,MK2718,"Giảm 7,000 VNĐ/1 thùng Dasani 510ML (24 chai)-...",6967,4753.0,Coca Cola,8.0,8.0,24.0,vn,67000.0,536000.0,"[""disabled products/productos desactivados"",""Đ...",Dasani,VND


In [9]:
print('before: ', supplier_order_df.shape[0])
supplier_order_df = supplier_order_df.drop_duplicates().reset_index(drop=True)
print('after: ', supplier_order_df.shape[0])

before:  2606440
after:  1356021


In [10]:
#supplier_order_df['category'] = supplier_order_df['category'].apply(lambda x: json.loads(x) if pd.notnull(x) else x)

In [11]:
supplier_order_df.head(3)

,supplier_order_id,created_at,delivery_lon,customer_id,delivery_lat,product_id,ean,name,brand_id,category_id,manufacturer,quantity,partial_quantity,units,country_id,price,amount,category,brand_name,currency
0,2637281,2022-11-15 01:18:50,106.700533,408215,10.79484,106912,58934588193052,Thùng Nước Cam Ép Twister 24 Chai x 455ML,5324,4752.0,PepsiCo,1.0,1.0,1.0,vn,188000.0,188000.0,"[""Nước trái cây"",""Đồ uống không cồn"",""Đồ uống""]",Twister,VND
1,2637281,2022-11-15 01:18:50,106.700533,408215,10.79484,109169,18935001214233,Thùng Trà bí đao WONDERFARM 310ML X 24 LON,5327,4754.0,Interfood,2.0,2.0,1.0,vn,146000.0,292000.0,"[""Trà"",""Đồ uống không cồn"",""Đồ uống""]",Wonderfarm,VND
2,2637281,2022-11-15 01:18:50,106.700533,408215,10.79484,109202,18935049501623,Thùng Sữa Trái Cây Nutriboost Hương Dâu 24 Cha...,5382,4767.0,Coke Marketplace,1.0,1.0,1.0,vn,171000.0,171000.0,"[""Sữa nước"",""Sữa và sản phẩm làm từ sữa""]",Nutriboost,VND


In [12]:
supplier_order_df[supplier_order_df['ean']=='049000050103']

,supplier_order_id,created_at,delivery_lon,customer_id,delivery_lat,product_id,ean,name,brand_id,category_id,manufacturer,quantity,partial_quantity,units,country_id,price,amount,category,brand_name,currency
3079,2329283,2022-09-20 20:22:47,-118.182313,376805,34.114271,49684,049000050103,Coke Classic 2 L,7943,2332.0,Coca Cola,5.0,5.0,8.0,us,17.29,86.45,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
8247,2382990,2022-09-29 20:08:20,-118.344145,123223,34.039458,49684,049000050103,Coke Classic 2 L,7943,2332.0,Coca Cola,3.0,3.0,8.0,us,16.89,50.67,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
19747,2623771,2022-11-11 20:24:30,-118.053995,392002,34.161672,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,Coca Cola,1.0,1.0,8.0,us,16.49,16.49,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
23111,2636898,2022-11-14 20:20:12,-117.915700,86414,33.781202,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,Coca Cola,4.0,4.0,8.0,us,16.89,67.56,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
25599,2441008,2022-10-12 04:09:37,-118.424551,289329,34.200806,49684,049000050103,Coke Classic 2 L,7943,2332.0,Coca Cola,10.0,10.0,8.0,us,16.89,168.90,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336644,2354653,2022-09-25 22:05:54,-118.131254,30695,34.011876,49684,049000050103,Coke Classic 2 L,7943,2332.0,Coca Cola,2.0,2.0,8.0,us,16.89,33.78,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
1338478,2598585,2022-11-09 04:38:29,-118.257185,84225,34.140866,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,Coca Cola,3.0,3.0,8.0,us,16.49,49.47,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
1346852,2375087,2022-09-28 17:05:23,-118.369958,262001,34.215337,49684,049000050103,Coke Classic 2 L,7943,2332.0,Coca Cola,20.0,20.0,8.0,us,16.89,337.80,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
1347671,2636798,2022-11-14 18:58:09,-118.177385,86289,33.898934,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,Coca Cola,10.0,10.0,8.0,us,16.89,168.90,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD


In [13]:
#supplier_order_df['category_name'] = supplier_order_df['category'].apply(lambda x:x[2] if pd.notnull(x) else x)

In [14]:
supplier_order_df = supplier_order_df.rename(columns={"delivery_lon": "lon","delivery_lat":"lat"})

In [15]:
#acá queda "created_at" queda del tipo string (para sacarle la hora) abajo lo casteo al type timestamp
supplier_order_df['created_at'] = supplier_order_df['created_at'].apply(lambda x: pd.Timestamp(pd.Timestamp(x).strftime("%Y-%m-%d")))

In [16]:
supplier_order_df['created_at'] = supplier_order_df['created_at'].apply(lambda x:pd.Timestamp(x))

In [17]:
#acá voy a flaguear para saber a que tab pertenece el registro
supplier_order_df.loc[(supplier_order_df['created_at']<=pd.Timestamp(start + timedelta(days =30))),'tab'] = '-30 to -60'
supplier_order_df.loc[(supplier_order_df['created_at']>pd.Timestamp(start + timedelta(days =30))),'tab'] = '0 to-30'

In [18]:
#esto para descartar si quedó alguno en null/nan
supplier_order_df = supplier_order_df[supplier_order_df['created_at']==supplier_order_df['created_at']].reset_index(drop = True) 

In [19]:
bucket = 'wabi-ops-zones'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key='wabi2b-zones/wabi2b_zones.kml')
#hasta acá tengo las ciudades, debería matchearlas con la info que tengo
full2b_zones_kml = gpd.GeoDataFrame.from_file(io.BytesIO(obj['Body'].read()))
zonas_mp = gpd.GeoDataFrame()
for ciudad in [full2b_zones_kml]:
    zonas_mp = zonas_mp.append(ciudad,ignore_index=True)

supplier_order_df = maps_2b.zonify(supplier_order_df ,zonas=zonas_mp, column= 'city')

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_11612\2995802072.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas_mp = zonas_mp.append(ciudad,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppD

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:06:34.918226


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [20]:
supplier_order_df[supplier_order_df['ean']=='049000050103']

,supplier_order_id,created_at,lon,customer_id,lat,product_id,ean,name,brand_id,category_id,...,units,country_id,price,amount,category,brand_name,currency,tab,geometry,city
3079,2329283,2022-09-20,-118.182313,376805,34.114271,49684,049000050103,Coke Classic 2 L,7943,2332.0,...,8.0,us,17.29,86.45,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,-30 to -60,POINT (-118.18231 34.11427),Los Ángeles
8247,2382990,2022-09-29,-118.344145,123223,34.039458,49684,049000050103,Coke Classic 2 L,7943,2332.0,...,8.0,us,16.89,50.67,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,-30 to -60,POINT (-118.34414 34.03946),Los Ángeles
19747,2623771,2022-11-11,-118.053995,392002,34.161672,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,...,8.0,us,16.49,16.49,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,0 to-30,POINT (-118.05399 34.16167),Los Ángeles
23111,2636898,2022-11-14,-117.915700,86414,33.781202,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,...,8.0,us,16.89,67.56,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,0 to-30,POINT (-117.91570 33.78120),Los Ángeles
25599,2441008,2022-10-12,-118.424551,289329,34.200806,49684,049000050103,Coke Classic 2 L,7943,2332.0,...,8.0,us,16.89,168.90,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,-30 to -60,POINT (-118.42455 34.20081),Los Ángeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336644,2354653,2022-09-25,-118.131254,30695,34.011876,49684,049000050103,Coke Classic 2 L,7943,2332.0,...,8.0,us,16.89,33.78,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,-30 to -60,POINT (-118.13125 34.01188),Los Ángeles
1338478,2598585,2022-11-09,-118.257185,84225,34.140866,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,...,8.0,us,16.49,49.47,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,0 to-30,POINT (-118.25718 34.14087),Los Ángeles
1346852,2375087,2022-09-28,-118.369958,262001,34.215337,49684,049000050103,Coke Classic 2 L,7943,2332.0,...,8.0,us,16.89,337.80,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,-30 to -60,POINT (-118.36996 34.21534),Los Ángeles
1347671,2636798,2022-11-14,-118.177385,86289,33.898934,49684,049000050103,Coca-Cola Classic 2 l,7943,2332.0,...,8.0,us,16.89,168.90,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD,0 to-30,POINT (-118.17739 33.89893),Los Ángeles


In [21]:
rate_query = '''
SELECT
er.rate_date as created_at,
er.currency_rate as currency,
er.rate 
FROM exchangeRate.exchange_rate er 
WHERE rate_date >="'''+date_time_str+'''"
'''

In [22]:
rate_df = sql_wabilytics.querySQL(rate_query,'exchangeRate')

In [23]:
#acá casteo el created_at a timestamp para poder hacer el merge tranqui con so
rate_df['created_at'] = rate_df['created_at'].apply(lambda x:pd.Timestamp(x))

In [24]:
rate_df.head(2)

,created_at,currency,rate
0,2022-09-18,AED,3.673100
1,2022-09-18,AFN,87.334632


In [25]:
supplier_order_df = supplier_order_df.merge(rate_df, how = 'left', on = ['created_at','currency'])

In [26]:
supplier_order_df['amount_usd'] = supplier_order_df['amount'] / supplier_order_df['rate']

In [27]:
supplier_orders_grouped = supplier_order_df.groupby(['tab','category','brand_name','product_id','ean','units','name','brand_id','category_id','manufacturer'
                           ,'city','country_id'], as_index=False).agg(stores_w_purchase=('customer_id','nunique'),
                                                                                       total_quantity =('quantity','sum'),
                                                                                       gmv=('amount','sum'),
                                                                                        gmv_usd =('amount_usd','sum'))

In [28]:
supplier_orders_grouped = supplier_orders_grouped.rename(columns = {'product_id':'Product ID','ean':'EAN','units':'Display','name':'Name','brand_id':'Brand ID',
                                    'brand_name':'Brand Name', 'category_id':'Category ID','manufacturer':'Manufacturer','city':'City',
                                     'country_id':'Country','stores_w_purchase':'Unique Stores With Purchase','category':'Category',
                                                                   'total_quantity':'Units Sold','gmv':'GMV (LC)',
                                                                   'gmv_usd':'GMV (USD)'})

In [29]:
df_0_30 = supplier_orders_grouped[supplier_orders_grouped['tab'] =='0 to-30']
df_0_30 = df_0_30[['Product ID','EAN','Display','Name','Brand ID','Brand Name','Category ID','Category','Manufacturer','City','Country',
                    'GMV (LC)','GMV (USD)','Units Sold','Unique Stores With Purchase']]

In [30]:
df_30_60 = supplier_orders_grouped[supplier_orders_grouped['tab'] =='-30 to -60']
df_30_60 = df_30_60[['Product ID','EAN','Display','Name','Brand ID','Brand Name','Category ID','Category','Manufacturer','City','Country',
                    'GMV (LC)','GMV (USD)','Units Sold','Unique Stores With Purchase']]

In [31]:
#guardo el df de 0-30
file = gSheets.readSheetByKey('1DBc3nLd-15PlHqCHRxDkHXoVWYTmcVoIk9MhMdtuBNI')
stores_wsheet = file.worksheet('0 to -30 days')
stores_wsheet.resize(rows = 2)
set_with_dataframe(stores_wsheet, df_0_30)

In [32]:
#guardo el df de 30-60
file = gSheets.readSheetByKey('1DBc3nLd-15PlHqCHRxDkHXoVWYTmcVoIk9MhMdtuBNI')
stores_wsheet = file.worksheet('-30 TO -60 days')
stores_wsheet.resize(rows = 2)
set_with_dataframe(stores_wsheet, df_30_60)

# Prueba

In [33]:
query = '''
SELECT
so.supplier_order_id,
so.created_at,
so.delivery_lon,
so.customer_id,
so.delivery_lat,
soi.product_id,
soi.ean,
soi.name,
soi.brand_id,
soi.category_id,
soi.manufacturer,
soi.quantity,
soi.partial_quantity,
soi.units,
s.country_id,
soi.price,
soi.amount,
pd.category, 
b.brand_name,
c.currency
FROM wabi2b.supplier_order so
LEFT JOIN wabi2b.suppliers s 
ON so.supplier_id = s.supplierId 
LEFT JOIN wabi2b.supplier_order_items soi
ON soi.supplier_order_id = so.supplier_order_id
LEFT JOIN wabi2b.country c
ON c.country_id = s.country_id 
LEFT JOIN wabi2b.product_details pd
ON soi.product_id = pd.product_id 
LEFT JOIN wabi2b.brand b 
ON pd.brand_id = b.brand_id
WHERE so.canceled_at IS NULL and so.created_at >='2022-09-15'
AND soi.ean = '049000050103'
'''

In [34]:
test_query = sql_wabilytics.querySQL(query,'wabi2b')

In [35]:
test_query

,supplier_order_id,created_at,delivery_lon,customer_id,delivery_lat,product_id,ean,name,brand_id,category_id,manufacturer,quantity,partial_quantity,units,country_id,price,amount,category,brand_name,currency
0,2329283,2022-09-20 20:22:47,-118.182313,376805,34.114271,49684,049000050103,Coke Classic 2 L,7943,2332,Coca Cola,5,5.0,8,us,17.29,86.45,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
1,2329283,2022-09-20 20:22:47,-118.182313,376805,34.114271,49684,049000050103,Coke Classic 2 L,7943,2332,Coca Cola,5,5.0,8,us,17.29,86.45,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
2,2382990,2022-09-29 20:08:20,-118.344145,123223,34.039458,49684,049000050103,Coke Classic 2 L,7943,2332,Coca Cola,3,3.0,8,us,16.89,50.67,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
3,2382990,2022-09-29 20:08:20,-118.344145,123223,34.039458,49684,049000050103,Coke Classic 2 L,7943,2332,Coca Cola,3,3.0,8,us,16.89,50.67,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
4,2441008,2022-10-12 04:09:37,-118.424551,289329,34.200806,49684,049000050103,Coke Classic 2 L,7943,2332,Coca Cola,10,10.0,8,us,16.89,168.90,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,2375087,2022-09-28 17:05:23,-118.369958,262001,34.215337,49684,049000050103,Coke Classic 2 L,7943,2332,Coca Cola,20,20.0,8,us,16.89,337.80,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
802,2507991,2022-10-25 01:44:27,-118.197606,81901,33.896367,49684,049000050103,Coca-Cola Classic 2 l,7943,2332,Coca Cola,13,13.0,8,us,16.89,219.57,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
803,2507991,2022-10-25 01:44:27,-118.197606,81901,33.896367,49684,049000050103,Coca-Cola Classic 2 l,7943,2332,Coca Cola,13,13.0,8,us,16.89,219.57,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD
804,2638552,2022-11-15 04:20:50,-118.344145,123223,34.039458,49684,049000050103,Coca-Cola Classic 2 l,7943,2332,Coca Cola,1,1.0,8,us,16.89,16.89,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,USD


In [36]:
print('before: ', test_query.shape[0])
test_query = test_query.drop_duplicates().reset_index(drop=True)
print('after: ', test_query.shape[0])

before:  806
after:  229


In [37]:
supplier_orders_grouped = test_query.groupby(['category','brand_name','product_id','ean','units','name','brand_id','category_id','manufacturer'
                           ,'country_id'], as_index=False).agg(
                                                                                       
                                                                                       gmv=('amount','sum')
                                                                                        )

In [38]:
supplier_orders_grouped

,category,brand_name,product_id,ean,units,name,brand_id,category_id,manufacturer,country_id,gmv
0,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,49684,049000050103,8,Coca-Cola Classic 2 l,7943,2332,Coca Cola,us,12508.78
1,"[""Soda"",""Non-Alcoholic Beverages"",""Beverages""]",Coca-Cola,49684,049000050103,8,Coke Classic 2 L,7943,2332,Coca Cola,us,12862.67
